# Install dependencies


In [ ]:
!pip install -U tk tensorrt PyPDF2 tqdm ipywidgets spacy

In [ ]:
!python -m spacy download it_core_news_lg

# Pre-training

In [18]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import sys

def writefile(filename, text):
    with open(filename, "w" ) as f:
        f.write(text)
    f.close
    return text

def readfile(filename):
    with open(filename, "r" ) as f:
        text = f.read()
    f.close
    return text

def readjson(filename):
    with open(filename, "r" ) as f:
        data = json.load(f)
    return data

def writejson(filename, data):
    with open(filename, "w" ) as f:
        f.write(json.dump(data))
    return True

def readbyte(filename):
    with open(filename, "rb") as f:
        data = f.read
    f.close
    return data

def writebyte(filename, byte):
    with open(filename, "wb" ) as f:
        f.write(byte)
    f.close
    return byte

# Estraggo il nome del file dall'input o dalla riga di comando
# aggiungo alla stringa l'estensione .pdf o .json
# ask 0 = file generico
# ask 1 = modulo (detector.py)
# ask 2 = training file [.json] (setup_pretrain.py)
# ask 3 = docfile [.spacy]
def argvfilename(filename, type="none", ask = "0"):
    if len(sys.argv) > 1:
        filename = sys.argv[1]
    else:
        if filename == "":
            filename = input(ask)
        else:
            filename=filename

        if type != "none":
            if type not in filename:
                if "." not in filename[:-1]:
                    filename = filename + "." + type
            else:
                    filename = filename + type
        return filename

# Estraggo eventualmente il numero della pagina dall'input o dalla riga di comando
# aggiungo alla stringa l'estensione .pdf
# Default "all" --> ?! inutile ?!
def argvpagenum(pagenum="all"):
    if len(sys.argv) > 2:
        pagenum = sys.argv[2]
    return pagenum

In [22]:
import spacy
from spacy.tokens import DocBin
import json

def getpretrain(trainfilename):
    nlp = spacy.blank("it") # Carico il modello spacy
    db = DocBin() # Creo l'oggetto DocBin
    with open("./models/" + trainfilename) as f:
        print("Apro " + trainfilename + "\n")
        TRAIN_DATA = json.load(f)

    # Raccolgo le informazioni su ogni dato usando la notazione Span di SpaCy
    for annotation in TRAIN_DATA:
        text = annotation["data"]["text"]
        entities = annotation["annotations"][0]["result"]
        doc = nlp(text)
        ents = []
        for entity in entities:
            start = entity["value"]["start"]
            end = entity["value"]["end"]
            label = entity["value"]["labels"][0]
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    output_name = trainfilename[0:-4]+"spacy"
    db.to_disk("./models/"+ output_name) # Salvo l'oggetto DocBin
    print("File .spacy creato")
    return output_name

In [23]:
trainfile = "data2.json"


In [25]:
spacyfile = getpretrain(trainfile)

Apro training_label.json

Skipping entity
File .spacy creato


In [ ]:
# CONVERTO DA CONLL
%spacy convert ./$trainfile -c conll .

In [26]:
!python -m spacy init config tools/config.cfg --lang it --pipeline ner --optimize accuracy -F

ℹ Generated config template specific for your use case
- Language: it
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
tools/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


# TRAINING

In [29]:
!python -m spacy train ./tools/config.cfg --output ./models --paths.train ./models/$spacyfile --paths.dev ./models/$spacyfile

ℹ Saving to output directory: models
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.48    0.00    0.00    0.00    0.00
  1     200        958.66   1592.17    0.00    0.00    0.00    0.00
  2     400        821.65    937.02   12.72   12.86   12.59    0.13
  3     600       1280.17    924.49   23.85   34.67   18.18    0.24
  5     800       2354.20    976.99   12.17   13.33   11.19    0.12
  7    1000       1540.97    882.34   19.35   19.85   18.88    0.19
  9    1200       2478.99   1363.18   40.14   39.07   41.26    0.40
 11    1400      10615.28   1881.97   29.54   37.23   24.48    0.30
 14    1600       7520.39   236

# READ & WRITE

In [37]:
import tkinter as tk
from tkinter import filedialog
import PyPDF2

def deletenwl(text):
    textlist = text.splitlines(True)
    #print(textlist)
    for i in range(len(textlist)):
        if textlist[i].endswith("-\n"):
            textlist[i] = textlist[i][0:-2]
        if textlist[i].endswith(".\n") :
            pass
        elif textlist[i].endswith("\n"):
            textlist[i] = textlist[i][0:-1]
    #print(textlist)
    extext = ' '.join(textlist)

    return extext


# Estrae il testo dal pdf e ritorna una stringa.
# E' possibile estrarre una pagina specifica modificando il parametro "pagine" (default "all")
def getPDFtext(PDFname, pagine="all", refine = False):
    # Creo una stringa vuota (extended text)
    extext = ""

    # Creo l'oggetto PDF
    pdfFileObj = open("./" + PDFname, 'rb')

    # Creo il lettore
    document = PyPDF2.PdfReader(pdfFileObj, strict = True)

    # Ottengo il numero delle pagine
    paginenum = len(document.pages)

    # Chiedo all'utente se desidera analizzare una pagina specifica o l'intero file o una slice
    # (se il fie è maggiore di una pagina)
    if paginenum > 1:
        print("Il documento contiene", paginenum, "pagine")
        scelta = pagine

            # Se sceglie l'intero documento
        if scelta == "all":
            for page in document.pages:
                pagetext = page.extract_text()
                extext = extext + pagetext

            print("Testo completo estratto con successo \n")

        elif ":" in scelta and len(scelta) >= 3:
            strt = ""
            end = ""
            for i2 in range(len(scelta)):
                if scelta[i2] == ":":
                    strt = scelta[0:i2]
                    end = scelta[i2+1:]

            p1 = int(strt)
            p2 = int(end)

            # Cumulo le pagine
            for page in document.pages[p1-2:p2]:
                pagetext = page.extract_text()
                extext = extext + pagetext

            print("Testo estratto con successo (da pag. " + str(p1) + " a pagina " + str(p2) + ")\n")

        # Se sceglie una pagina specifica
        else:
            # Trasformo l'input in int e sottraggo per far corrispondere l'indice
            scelta = int(scelta)
            pagineObj = document.pages[scelta]
            text = pagineObj.extractText()
            extext = extext + text

            print("Testo di pagina " + str(scelta) + " estratto con successo \n")
    else:
        # Estraggo la singola pagina
        pagineObj = document.pages[0]
        text = pagineObj.extractText()
        extext = text[0]

        print("Testo estratto con successo \n")

    # Elimino le newlines
    #extext = deletenwl(extext)

    # Chiudo file PDF
    pdfFileObj.close()
    return deletenwl(extext)


# Estraggo il testo con getPDFtext() e creo un file PDFtext.txt temporaneo nella cartella output, ritorna il nome del file come stringa
def writePDFtxt(PDFname, filename, pagine="all"):

    PDFtext = getPDFtext(PDFname, pagine)

    if filename == "":
        filename = PDFname[0:-4]
    with open("output/" + filename + ".txt", "w") as f:
        f.write(PDFtext)

    f.close
    return PDFname


#funzione principale che ottiene il nome del file .pdf e crea il file .txt
##def pdftotxt(PDFfilename = argvfilename("pdf"), pagine=argvpagenum()):

#writePDFtxt("codice_civile.pdf", "codice_civile")
#print("File .txt creato!\n")




ModuleNotFoundError: ignored

In [31]:
import spacy
from spacy.tokens import DocBin

#UNDERSTAND
# --> applica il modello al file di testo
def getDoc(filename, modelname = "model-last", textdata = "PDFtext.txt", path = "default"):

    if modelname == None or modelname == "":
        modelname = argvfilename("", "none", "Inserisci il nome del modello \n")

    if textdata == None or textdata == "":
        textdata = argvfilename("", "txt", "Inserisci il nome del file di testo da analizzare [.txt] \n")

    nlp_process = spacy.load("./models/" + modelname)

    text = readfile("output/" + textdata)

    if path == "default":
        path = "output/"+filename
    else:
        path = path
        if "/" not in filename[:-1]:
            filename = filename + "/"

    doc = nlp_process(text)
    print (filename + " terminato! \n")

    return doc

In [ ]:
## WRITE ##
PDFname = "esempio.pdf"
txtTitle = "sistema"
pages =  "all"

In [ ]:
writePDFtxt(PDFname, txtTitle, pages)

Il documento contiene 56 pagine
Testo completo estratto con successo 



'esempio.pdf'

In [34]:
## READ ##
process_name = "artdetector"
model = "model-last"
text = "sistema.txt"
path = "default"

In [41]:
doc_ = getDoc(process_name, model, text, path)

RegistryError: ignored

In [ ]:
## EXPlAIN ##
label = "DECRETO LEGISLATIVO"

In [ ]:
for i in range(len(doc_.ents)):
        if doc_.ents[i].label_ == label:
                print(doc_.ents[i])